In [1]:
from pythreejs import *

ImportError: No module named pythreejs

In [3]:
import numpy as np
from IPython.display import display
from traittypes import Array
from ipywidgets import HTML, Text, Widget
from traitlets import link, dlink, Any, HasTraits

In [4]:
class Surf(HasTraits):
    
    function = Any()
    points = Array()
    
    def __init__(self, **kwargs):
        self.xmin = kwargs.pop('xmin', -1)
        self.xmax = kwargs.pop('xmax', 1)
        self.ymin = kwargs.pop('ymin', -1)
        self.ymax = kwargs.pop('ymax', 1)
        
        self.grid_size = kwargs.pop('grid_size', 20)
        
        self.points = kwargs.pop('points', np.zeros((1, 3)))
        self.observe(self.update_points, 'points')
            
        self.point_size = kwargs.pop('size', 0.05)
        
        self.function = kwargs.pop('function', lambda xx, yy: xx**2 - yy**2)
        self.observe(self.update_surf, 'function')
        
        self.surf_color = kwargs.pop('surf_color', 'viridis')
        self.line_color = kwargs.pop('line_color', 'black')
        self.point_colors = kwargs.pop('point_colors', ['red'])
        self.cam_position = [0, 3, 3]
        self.cam_up = [0, 0, 1]
        
        self.plot(**kwargs)
        self.draw(**kwargs)
        
    def plot(self, **kwargs):
        x = np.linspace(self.xmin, self.xmax, self.grid_size)
        y = np.linspace(self.xmin, self.xmax, self.grid_size)
        xx, yy = np.meshgrid(x, y)
        self.z = self.function(xx, yy)
        
    def update_surf(self, new):
        self.plot()
        self.surface_geom.z = list(self.z[::-1].flat)
        self.surface.material = LambertMaterial(map=height_texture(self.z[::-1], self.surf_color))
        
    def update_points(self, new):
        self.draw_points()
        self.marks = [self.surface, self.surfgrid] + self.dots + [AmbientLight(color='#777777')]
        self.scene.children = self.marks
        
    def draw_surf(self, **kwargs):
        self.surface_geom = SurfaceGeometry(z=list(self.z[::-1].flat), 
                         width=2 * self.xmax,
                         height=2 * self.xmax,
                         width_segments=self.grid_size - 1,
                         height_segments=self.grid_size - 1)
        self.surface = Mesh(geometry=self.surface_geom, 
                         material=LambertMaterial(map=height_texture(self.z[::-1], self.surf_color)))
        self.surfgrid = SurfaceGrid(geometry=self.surface_geom, material=LineBasicMaterial(color=self.line_color))
        
    def draw_points(self, **kwargs):
        self.dots = []
        for i in range(self.points.shape[0]):
            self.dots.append(Mesh(geometry=SphereGeometry(radius=self.point_size), 
                                    material=LambertMaterial(color=self.point_colors[i % len(self.point_colors
                                                                                            )]),
                                    position=self.points[i, :].tolist()))
    
    def draw(self, **kwargs):
        self.draw_surf(**kwargs)
        self.draw_points(**kwargs)
        
        self.marks = [self.surface, self.surfgrid] + self.dots + [AmbientLight(color='#777777')]
        
        self.scene = Scene(children=self.marks)
        self.cam = PerspectiveCamera(position=self.cam_position, up=self.cam_up, 
                              children=[DirectionalLight(color='white', position=[3, 5, 1], intensity=0.6)])

        self.figure = Renderer(camera=self.cam,
                                 scene = self.scene, controls=[OrbitControls(controlling=self.cam)])
        return

In [5]:
a = Surf()
a.figure

In [6]:
a.function = lambda x, y: x**3 - 3*x*(y**2)

In [7]:
a.points = np.array([[0, 0, 1], [0, 0, -1]])

In [8]:
from sympy import *

In [9]:
def evaluate_gradient(func, point, variables=['x', 'y']):
    var = []
    grad = []
    for i in variables:
        var.append(symbols(i))
        
    if len(point) != len(variables):
        raise ValueError('Length of points doesnt match the number of variables')

    for i in range(len(var)):
        diff_func = lambdify(variables, diff(sympify(func), var[i]))
        grad.append(diff_func(*point))
    return np.array(grad)

In [10]:
func = 'x**3 - 3*x*(y**2)'

In [11]:
evaluate_gradient(func, [2, 1])

array([  9, -12])

In [35]:
loc = [.2,.2]

In [36]:
z=a.function(loc[0],loc[1])

In [37]:
a.points = np.array([[loc[0],loc[1],z]])

In [39]:
g = evaluate_gradient(func, loc)

In [40]:
step_size = .1

In [41]:
loc = loc - step_size * g

In [43]:
a.points.append([0,0,1])

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [44]:
a.points.shape

(1, 3)